In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:\\Users\\ws-1\\Documents\\Tweets.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
df.shape

(27481, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [5]:
df.nunique()

textID           27481
text             27480
selected_text    22463
sentiment            3
dtype: int64

In [6]:
df.describe()

,textID,text,selected_text,sentiment
count,27481,27480,27480,27481
unique,27481,27480,22463,3
top,cb774db0d1,"I`d have responded, if I were going",good,neutral
freq,1,1,199,11118


In [7]:
df.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [8]:
df.dropna(inplace=True)



In [9]:
df=df.drop(['selected_text', 'textID'],axis=1) 
df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [10]:
print(df['sentiment'].value_counts())

sentiment
neutral     11117
positive     8582
negative     7781
Name: count, dtype: int64


In [11]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [12]:
# Assuming df is your DataFrame
sentiment_counts = df['sentiment'].value_counts()

# Print the counts of each sentiment class
print(sentiment_counts)

# Calculate the proportion of each sentiment class
proportions = sentiment_counts / sentiment_counts.sum()

# Print the proportions
print(proportions)

sentiment
neutral     11117
positive     8582
negative     7781
Name: count, dtype: int64
sentiment
neutral     0.404549
positive    0.312300
negative    0.283151
Name: count, dtype: float64


In [13]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

wordnet = WordNetLemmatizer()

def preprocess_sentence(sentence):
    
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)  # Remove non-alphabet characters
    sentence = sentence.lower()  # Convert to lowercase
    words = sentence.split()  # Split into words
    words = [wordnet.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]  # Lemmatize and remove stopwords
    return ' '.join(words)


# Apply the preprocessing function to the 'Commentst' column
df['text'] = df['text'].apply(preprocess_sentence)

In [14]:
df.head()

,text,sentiment
0,responded going,neutral
1,sooo sad miss san diego,negative
2,bos bullying,negative
3,interview leave alone,negative
4,son put release already bought,negative


In [15]:
# Replacing  'positive' with 1 and 'negative' with 0 in the 'sentiment' column
df['sentiment'] = df['sentiment'].replace({'positive':1, 'neutral':0, 'negative':-1})

In [16]:
df.head()

,text,sentiment
0,responded going,0
1,sooo sad miss san diego,-1
2,bos bullying,-1
3,interview leave alone,-1
4,son put release already bought,-1


In [17]:
y = df['sentiment']
x = df['text']

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25, random_state=42)

In [19]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((20610,), (6870,), (20610,), (6870,))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Define tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

# Convert text data to sequences
X_train_seq = tokenizer.texts_to_sequences(x_train)
X_test_seq = tokenizer.texts_to_sequences(x_test)

# Get the maximum sequence length
max_sequence_length = max(len(seq) for seq in X_train_seq + X_test_seq)

# Pad sequences to the same length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Define your LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.LSTM(1000, activation='relu', return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(500, activation='relu', return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(300, activation='relu', return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(100, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert class labels to one-hot encoding
y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
X_test_tfidf = tfidf_vectorizer.transform(x_test)

In [21]:
y_train_tfidf=y_train
y_test_tfidf=y_test

In [22]:
# Convert the SparseTensor objects to dense NumPy arrays
X_train_tfidf = X_train_tfidf.toarray()
X_test_tfidf = X_test_tfidf.toarray()

In [ ]:
import tensorflow as tf

# Define your LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, input_shape=(X_train_tfidf.shape[1], 1)),  # LSTM layer with 128 units
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),  # Dense layer with 64 neurons
    tf.keras.layers.Dropout(0.5),  # Dropout layer with dropout rate 0.5
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer with 3 neurons for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data for LSTM (assuming X_train_tfidf and X_test_tfidf are TF-IDF matrices)
X_train_lstm = X_train_tfidf.reshape(X_train_tfidf.shape[0], X_train_tfidf.shape[1], 1)
X_test_lstm = X_test_tfidf.reshape(X_test_tfidf.shape[0], X_test_tfidf.shape[1], 1)

# Convert class labels to one-hot encoding
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train_tfidf, num_classes=3)
y_test_encoded = to_categorical(y_test_tfidf, num_classes=3)

# Train the model
model.fit(X_train_lstm, y_train_encoded, epochs=10, validation_data=(X_test_lstm, y_test_encoded))

Epoch 1/10
227/645 [=========>....................] - ETA: 2:51:28 - loss: 1.0917 - accuracy: 0.3990

In [32]:
import imblearn
from imblearn.over_sampling import SMOTE

# Assuming x_train, y_train, x_test, y_test are already defined

# Initialize SMOTE
sm = SMOTE(random_state=42)

# Generating synthetic data points for training set
x_smote_train, y_smote_train = sm.fit_resample(X_train_tfidf, y_train_tfidf)

# Generating synthetic data points for testing set
x_smote_test, y_smote_test = sm.fit_resample(X_test_tfidf, y_test_tfidf)

In [33]:
# Count occurrences of each class in the oversampled training set
print("Oversampled Training Set:")
print(pd.Series(y_smote_train).value_counts())

# Count occurrences of each class in the oversampled testing set
print("\nOversampled Testing Set:")
print(pd.Series(y_smote_test).value_counts())

Oversampled Training Set:
sentiment
 0    8330
 1    8330
-1    8330
Name: count, dtype: int64

Oversampled Testing Set:
sentiment
 1    2787
-1    2787
 0    2787
Name: count, dtype: int64


In [34]:
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)


X_train_tfidf shape: (20610, 10000)
X_test_tfidf shape: (6870, 10000)


In [35]:
import tensorflow as tf

# Define your LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(1000, return_sequences=True, input_shape=(X_train_tfidf.shape[1], 1)),  # LSTM layer with 1000 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(500, return_sequences=True),  # LSTM layer with 500 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(300, return_sequences=True),  # LSTM layer with 300 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(100),  # LSTM layer with 100 units
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer with 3 neurons for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data for LSTM (assuming X_train_tfidf and X_test_tfidf are TF-IDF matrices)
X_train_lstm = X_train_tfidf.reshape(X_train_tfidf.shape[0], X_train_tfidf.shape[1], 1)
X_test_lstm = X_test_tfidf.reshape(X_test_tfidf.shape[0], X_test_tfidf.shape[1], 1)

# Convert class labels to one-hot encoding
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train_tfidf, num_classes=3)
y_test_encoded = to_categorical(y_test_tfidf, num_classes=3)

# Train the model
model.fit(X_train_lstm, y_train_encoded, epochs=10, validation_data=(X_test_lstm, y_test_encoded))

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node transpose_1 defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[32,10000,500] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node transpose_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[sequential_1/lstm_5/PartitionedCall]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_22957]

In [79]:
import tensorflow as tf
from keras.utils import to_categorical

# Define your LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(1000, return_sequences=True, input_shape=(X_train_tfidf.shape[1], 1)),  # LSTM layer with 1000 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(500, return_sequences=True),  # LSTM layer with 500 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(300, return_sequences=True),  # LSTM layer with 300 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(100),  # LSTM layer with 100 units
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer with 3 neurons for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data for LSTM
X_train_lstm = X_train_tfidf.reshape(X_train_tfidf.shape[0], X_train_tfidf.shape[1], 1)
X_test_lstm = X_test_tfidf.reshape(X_test_tfidf.shape[0], X_test_tfidf.shape[1], 1)

# Convert class labels to one-hot encoding
y_train_encoded = to_categorical(y_train_tfidf, num_classes=3)
y_test_encoded = to_categorical(y_test_tfidf, num_classes=3)

# Train the model
model.fit(X_train_lstm, y_train_encoded, epochs=10, validation_data=(X_test_lstm, y_test_encoded))


ValueError: matrix shape must be two-dimensional

In [ ]:
import tensorflow as tf
from keras.utils import to_categorical

# Define your LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(1000, return_sequences=True, input_shape=(X_train_tfidf.shape[1], 1)),  # LSTM layer with 1000 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(500, return_sequences=True),  # LSTM layer with 500 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(300, return_sequences=True),  # LSTM layer with 300 units and return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.LSTM(100),  # LSTM layer with 100 units
    tf.keras.layers.Dropout(0.2),  # Dropout layer with dropout rate 0.2
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer with 3 neurons for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data for LSTM
X_train_lstm = X_train_tfidf.reshape(X_train_tfidf.shape[0], X_train_tfidf.shape[1], 1)
X_test_lstm = X_test_tfidf.reshape(X_test_tfidf.shape[0], X_test_tfidf.shape[1], 1)

from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y_train_numerical = label_encoder.fit_transform(y_train_tfidf)
y_test_numerical = label_encoder.transform(y_test_tfidf)

# Convert class labels to one-hot encoding
y_train_encoded = to_categorical(y_train_tfidf, num_classes=3)
y_test_encoded = to_categorical(y_test_tfidf, num_classes=3)

# Train the model
model.fit(X_train_lstm, y_train_encoded, epochs=10, validation_data=(X_test_lstm, y_test_encoded))


In [84]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from sklearn.metrics import accuracy_score

# Assuming x_smote_train, y_smote_train, x_smote_test, y_smote_test are oversampled datasets

# Define LSTM model
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=3, input_length=X_train_tfidf.shape[1], 1)))
model.add(LSTM(units=100))  # LSTM layer with 100 units
model.add(Dense(units=1, activation='sigmoid'))  # Output layer, assuming binary classification


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_smote_train, y_smote_train, batch_size=64, epochs=10, validation_split=0.1)

# Evaluate the model on test data
y_pred = model.predict_classes(x_smote_test)
accuracy = accuracy_score(y_smote_test, y_pred)
print("Test Accuracy:", accuracy)


SyntaxError: positional argument follows keyword argument (684686983.py, line 10)